In [2]:
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

### Preprocessing of sp-dep-jour-2023-06-30-16h26.csv 
##### First part of the dashboard

In [3]:
data = pd.read_csv('../raw_data/sp-dep-jour-2023-06-30-16h26.csv', sep=';')
data.head()

,dep,jour,pop,P,T,Ti,Tp,Td,cl_age90
0,01,13/05/2020,656955,9,340,"1,37","2,65","51,75",0
1,01,14/05/2020,656955,9,440,"1,37","2,05","66,98",0
2,01,15/05/2020,656955,5,454,"0,76","1,1","69,11",0
3,01,16/05/2020,656955,0,151,0,0,"22,98",0
4,01,17/05/2020,656955,0,32,0,0,"4,87",0


In [4]:
for name in ['Ti','Td','Tp']:
    data[name] = data[name].str.replace(',', '.')
    data[name] = data[name].astype(float)

In [5]:
df = data.copy()
df = df.fillna(0)
df['date'] = pd.to_datetime(data['jour'])
df.drop(['dep','jour','cl_age90'], axis=1, inplace=True)
df = df.groupby('date').agg({'P':'sum', 'T':'sum','Ti':'mean', 'Tp':'mean','Td':'mean', 'pop':'sum'}).reset_index()
df.head(2)

C:\Users\samyj\AppData\Local\Temp\ipykernel_31792\558924983.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(data['jour'])


,date,P,T,Ti,Tp,Td,pop
0,2020-05-13,889,39287,1.135490,2.275000,54.462255,67099037
1,2020-05-14,1001,42653,1.284902,2.294608,59.428333,67099037


In [6]:
df['P7'] = df['P'].rolling(window=7).mean()
df['T7'] = df['T'].rolling(window=7).mean()

In [9]:
df.to_csv('../preprocessed_data/epidemic_state.csv', index=False)

#### Last page of the dashboard

In [8]:
critical_beds = pd.read_csv('../raw_data/critical_beds_dep.csv', sep=';')
critical_beds.head()

,Code,Libellé,2013,2019,2020,2021,2022
0,FR,France entière,10 767,11 377,12 203,11 994,11 664
1,MET,France métropolitaine,10 538,11 076,11 873,11 647,11 321
2,01,Ain,38,34,34,36,36
3,02,Aisne,90,89,96,92,84
4,03,Allier,50,52,58,50,46


Change layout of the df and type of data

In [9]:
critical_beds = critical_beds.melt(id_vars=['Code', "Libellé"], value_vars=["2013","2019", "2020", "2021", "2022"]).rename(columns={"Code":"dep","variable": "year", "value": "Critical beds"})
critical_beds['year'] = critical_beds['year'].astype(int)
critical_beds['Critical beds'] = critical_beds['Critical beds'].apply(lambda x: x.replace(" ", "")).astype(int)
critical_beds["dep"] = critical_beds["dep"].astype(str).apply(lambda x: x.zfill(2))

In [10]:
critical_beds.head()

,dep,Libellé,year,Critical beds
0,FR,France entière,2013,10767
1,MET,France métropolitaine,2013,10538
2,01,Ain,2013,38
3,02,Aisne,2013,90
4,03,Allier,2013,50


Covid data

In [4]:
covid_data = pd.read_csv('../raw_data/sursaud-covid19-departement.csv', sep=';', low_memory=False)

In [8]:
covid_data[covid_data["dep"]=="01"].nbre_acte_tot.sum()

0.0

In [12]:
covid_data["dep"] = covid_data["dep"].astype(str).apply(lambda x: x.zfill(2))
covid_data = covid_data.drop(columns=["sursaud_cl_age_corona"]).groupby(['dep','date_de_passage']).sum().reset_index()
covid_data['date_de_passage'] = pd.to_datetime(covid_data['date_de_passage'])
covid_data = covid_data.rename(columns={"date_de_passage":"date"})

In [13]:
covid_data.head()

,dep,date,nbre_pass_corona,nbre_pass_tot,nbre_hospit_corona,nbre_pass_corona_h,nbre_pass_corona_f,nbre_pass_tot_h,nbre_pass_tot_f,nbre_hospit_corona_h,nbre_hospit_corona_f,nbre_acte_corona,nbre_acte_tot,nbre_acte_corona_h,nbre_acte_corona_f,nbre_acte_tot_h,nbre_acte_tot_f
0,01,2020-02-24,0.0,714.0,0.0,0.0,0.0,202.0,155.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01,2020-02-25,0.0,620.0,0.0,0.0,0.0,177.0,133.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01,2020-02-26,0.0,732.0,0.0,0.0,0.0,193.0,173.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01,2020-02-27,0.0,642.0,0.0,0.0,0.0,178.0,143.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01,2020-02-28,0.0,678.0,0.0,0.0,0.0,166.0,173.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Covid data with focus on hospitalization

In [3]:
focus_hosp = pd.read_csv("../raw_data/covid-hospit-2023-03-31-18h01.csv", sep=';')

In [15]:
focus_hosp = focus_hosp[["dep","sexe", "jour", "hosp", "rea"]]
focus_hosp = focus_hosp.drop(columns=["sexe"]).groupby(['dep','jour']).sum().reset_index()
focus_hosp["jour"] = pd.to_datetime(focus_hosp["jour"])
focus_hosp = focus_hosp.rename(columns={"jour":"date"})

In [16]:
focus_hosp.head()

,dep,date,hosp,rea
0,01,2020-03-18,4,0
1,01,2020-03-19,4,0
2,01,2020-03-20,4,0
3,01,2020-03-21,8,0
4,01,2020-03-22,16,2


Merge the three datasets

In [17]:
df = pd.merge(covid_data, focus_hosp, on=["date",'dep'], how="left")

In [18]:
all_dep_data = df.drop(columns=["dep"]).groupby(['date']).sum().reset_index()
all_dep_data["dep"] = "FR"
df = pd.concat([df, all_dep_data])
df["year"] = df["date"].dt.year

In [19]:
df = pd.merge(df, critical_beds, on=["year","dep"], how="left")

Computes the main KPIs

In [20]:
# Proportion of emergency room visits for suspected COVID-19 by region compared with the number of emergency room visits for all reasons
df['emergency_hospitals'] = 100*df['nbre_pass_corona'] / df['nbre_pass_tot']
# 7-day moving average ignoring missing values   
df['emergency_hospitals'] = df['emergency_hospitals'].rolling(window=7, min_periods=1).mean()

In [21]:
# Proportion of emergency room visits for suspected COVID-19 by region compared with the number of emergency room visits for all reasons
df['SOS_med_call'] = 100*df['nbre_acte_corona'] / df['nbre_acte_tot']

# 7-day moving average ignoring missing values
df['SOS_med_call'] = df['SOS_med_call'].rolling(window=7, min_periods=1).mean()

In [22]:
# Proportion of intensive care beds occupied by COVID-19 patients
df['critical_care_beds'] = 100*df['rea'] / df['Critical beds']

# 7-day moving average ignoring missing values
df['critical_care_beds'] = df['critical_care_beds'].rolling(window=7, min_periods=1).mean()

In [23]:
df = df.rename(columns={"SOS_med_call":"Share of SOS med calls for Covid",
                        "emergency_hospitals":"Share of hospital emergency visits for Covid",
                        "critical_care_beds":"Share of all critical care beds occupied by Covid patients"})

In [24]:
df.tail(10)

,dep,date,nbre_pass_corona,nbre_pass_tot,nbre_hospit_corona,nbre_pass_corona_h,nbre_pass_corona_f,nbre_pass_tot_h,nbre_pass_tot_f,nbre_hospit_corona_h,...,nbre_acte_tot_h,nbre_acte_tot_f,hosp,rea,year,Libellé,Critical beds,Share of hospital emergency visits for Covid,Share of SOS med calls for Covid,Share of all critical care beds occupied by Covid patients
35384,FR,2021-01-26,2162.0,69614.0,1260.0,573.0,508.0,17739.0,17067.0,363.0,...,3855.0,5000.0,53713.0,6109.0,2021,France entière,11994,3.086676,4.428121,48.691012
35385,FR,2021-01-27,2036.0,67821.0,1136.0,537.0,481.0,17396.0,16513.0,326.0,...,3920.0,5028.0,53960.0,6164.0,2021,France entière,11994,3.095192,4.368159,49.300841
35386,FR,2021-01-28,2270.0,72317.0,1368.0,544.0,590.0,18321.0,17832.0,368.0,...,3851.0,5170.0,53949.0,6171.0,2021,France entière,11994,3.096330,4.394749,49.865409
35387,FR,2021-01-29,2278.0,72663.0,1376.0,558.0,581.0,18213.0,18110.0,374.0,...,3619.0,4938.0,54227.0,6209.0,2021,France entière,11994,3.069238,4.297119,50.385907
35388,FR,2021-01-30,1992.0,66485.0,1160.0,515.0,481.0,16968.0,16273.0,318.0,...,4599.0,6142.0,54167.0,6173.0,2021,France entière,11994,3.053170,4.319019,50.900450
35389,FR,2021-01-31,1956.0,63428.0,1104.0,502.0,476.0,16261.0,15449.0,298.0,...,4688.0,6138.0,54826.0,6266.0,2021,France entière,11994,3.081533,4.403334,51.362586
35390,FR,2021-02-01,2564.0,79597.0,1476.0,665.0,617.0,20513.0,19283.0,416.0,...,4390.0,5935.0,55429.0,6408.0,2021,France entière,11994,3.097557,4.422325,51.811620
35391,FR,2021-02-02,2178.0,72175.0,1312.0,574.0,515.0,18539.0,17545.0,377.0,...,4049.0,5468.0,55763.0,6518.0,2021,France entière,11994,3.084981,4.393090,52.298768
35392,FR,2021-02-03,2170.0,68715.0,1282.0,545.0,540.0,17684.0,16669.0,352.0,...,4013.0,5314.0,55618.0,6510.0,2021,France entière,11994,3.107260,4.411889,52.710879
35393,FR,2021-02-04,1622.0,59439.0,862.0,410.0,401.0,15303.0,14412.0,237.0,...,3922.0,5231.0,55234.0,6455.0,2021,France entière,11994,3.048673,4.419123,53.049144


In [26]:
df.to_csv("../preprocessed_data/covid19-saturation-dep.csv", index=False)